# Flask and SQL

### Introduction

In the last lesson, we saw how we could generate different web responses based on the user's input through a url.  This was our code:

```python
from flask import Flask, render_template
app = Flask(__name__, static_folder='public', template_folder='views')
  
@app.route('/nhl/players/<name>')
def show_player(name):
    return render_template('player.html', name=name)
```

Now of course, this is fairly simplistic.  The user provides the players name, and the website responds with the player's name.  In this lesson, we'll see how the website can respond with more information that it has stored in the SQL database.

### Getting our Data

We can quickly get some data and load it into a database with the following.

In [39]:
import pandas as pd
df = pd.read_csv('./nhl_players.csv')

In [36]:
import sqlite3
conn = sqlite3.connect('nhl.db')

In [37]:
df.to_sql('players', conn, index = False)

In [42]:
cursor = conn.cursor()

In [44]:
cursor.execute('select * from players where id = 0;')
cursor.fetchone()

(0,
 'Connor Bunnaman',
 21,
 '6\' 1"',
 '207 lbs',
 'L',
 'Guelph, ON',
 '04/16/98',
 82)

So, now this is the data that we can then present to the user.  The user can type type in (or click on a link to a url like) `/nhl/players/0`, and then we will make a SQL call to select the players with the id equal to 0.

### Seeing it in Action

```python
# application.py
import sqlite3
from flask import Flask, render_template

app = Flask(__name__, static_folder='public', template_folder='views')

@app.route('/nhl/players/<player_id>')
def show_player(player_id):
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players where id = ?;', player_id)
    player_details = cursor.fetchone()
    return str(player_details)

app.run()
```

Then from the command line run the following.

`python application.py`

We'll see different player information appear based on the url we visit.  Take a moment to think about how this works.  Each time a different the user visits a url with a different id at the end, sql will select information from a different player to display.

One way to see this is to simply copy and paste the `show_player` function, and use it on it's own.

In [52]:
def show_player(player_id):
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players where id = ?;', player_id)
    player_details = cursor.fetchone()
    return str(player_details)

In [53]:
show_player('1')

'(1, \'Sean Couturier\', 27, \'6\\\' 3"\', \'211 lbs\', \'L\', \'Phoenix, AZ\', \'12/07/92\', 14)'

In [54]:
show_player('2')

'(2, \'Kevin Hayes\', 27, \'6\\\' 5"\', \'216 lbs\', \'L\', \'Boston, MA\', \'05/08/92\', 13)'

So an `id` like `'2'` comes the url when the user types in `\nhl\players\2`, and then this id is passed to the `show_player` function.

### Adding Objects

Before we add in some HTML, there is still the fact that working with objects is simply better, and easier.  So immediately after we select the player's information from SQL, we can then create a player instance with this information.  Of course to do that, we first need to create a Player class, and then import it into our `application.py` file.  Let's do that.  We'll create a `player.py` file, where we'll define a Player class.

In [51]:
class Player:
    def __init__(self, id, name, age, height, weight, shot, birth_place, birthdate, number):
        self.id = id
        self.name = name
        self.age = age
        self.height = height
        self.weight = weight
        self.shot = shot
        self.birth_place = birth_place
        self.birthdate = birthdate
        self.number = number

Then we can update the `show_player` function so that after selecting the information of the player from the database, it initializes an instance of a player, with that information.

We'll get `player_info` like the following:

In [60]:
player_info = (2, 'Kevin Hayes', 27, '6\' 5"', '216 lbs', 'L', 'Boston, MA', '05/08/92', 13)

And now we need to initialize an object with this information.

One way is to specify all of the arguments.

In [64]:
hayes = Player(id = player_info[0], name = player_info[1],
       age = player_info[2], height = player_info[3],
       weight = player_info[4], shot = player_info[5],
       birth_place = player_info[6], birthdate = player_info[7],
       number = player_info[8]
      )

In [65]:
hayes.__dict__

{'id': 2,
 'name': 'Kevin Hayes',
 'age': 27,
 'height': '6\' 5"',
 'weight': '216 lbs',
 'shot': 'L',
 'birth_place': 'Boston, MA',
 'birthdate': '05/08/92',
 'number': 13}

Another way is to take our `player_info` and use * to initialize the instance.

In [ ]:
player_info = (2, 'Kevin Hayes', 27, '6\' 5"', '216 lbs', 'L', 'Boston, MA', '05/08/92', 13)

In [66]:
hayes = Player(*player_info)

In [68]:
hayes.__dict__

{'id': 2,
 'name': 'Kevin Hayes',
 'age': 27,
 'height': '6\' 5"',
 'weight': '216 lbs',
 'shot': 'L',
 'birth_place': 'Boston, MA',
 'birthdate': '05/08/92',
 'number': 13}

Ok, let's use the second way because it's easier.  We'll update our `application.py` file to load in the Player class, and initialize a new player with the data retrieved from the database.

```python
# application.py
import sqlite3
from flask import Flask, render_template
from player import Player

app = Flask(__name__, static_folder='public', template_folder='views')

@app.route('/nhl/players/<player_id>')
def show_player(player_id):
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players where id = ?;', player_id)
    player_details = cursor.fetchone()
    player = Player(*player_details)
    return str(player.__dict__)

app.run()
```

<img src="./sean-dict.png">

<img src="./travis.png">

### Adding HTML

Finally, we can add in some HTML to our website.  We can update our `show_player` function to render the `player.html` file.

In [ ]:
# application.py
import sqlite3
from flask import Flask, render_template
from player import Player

app = Flask(__name__, static_folder='public', template_folder='views')

@app.route('/nhl/players/<player_id>')
def show_player(player_id):
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players where id = ?;', player_id)
    player_details = cursor.fetchone()
    player = Player(*player_details)
    return render_template('player.html',  player = player)

app.run()

And then in our `views/player.html` file we have the following:

```html
Welcome to the webpage of 
<h1> {{player.name}}</h1>
```

### Updating our HTML

Finally, we copied and pasted some of the HTML from ESPN for you.  We did so by simply visiting [the following](view-source:http://www.espn.com/nhl/player/_/id/2562601) and pasting it into a file called `views/player-1.html`.

Then if you look at line 691 of the file, you can see that we modified some of the HTML with our special `{{player.name}}`.  So now if you update the `show_player` method to render this file, you should see ESPN's player page with different player information from your website.

```python
def show_player(player_id):
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players where id = ?;', player_id)
    player_details = cursor.fetchone()
    player = Player(*player_details)
    return render_template('player-1.html',  player = player)
```

All we're doing is changing the player's name.  We'll leave it to you to make some of the other information that we have for a player dynamic.

### Summary